# Triple-Extraktor für frühneuzeitliche Briefe

**Version:** 1.0.0.0

Dieses Notebook extrahiert semantische Triples aus frühneuzeitlichen Briefen mit Fokus auf Konzept-Hierarchien.

In [ ]:
# Abhängigkeiten installieren (einmal ausführen)
!pip install -q plotly networkx kaleido numpy matplotlib

In [ ]:
# Setup (einmal ausführen)
# Installiert benötigte Pakete und lädt den Code.

# Version
NOTEBOOK_VERSION = "1.0.0.0"
print(f"📓 Triple-Extraktor Notebook v{NOTEBOOK_VERSION}")
print(f"   Letzte Aktualisierung: 07.02.2026\n")

import sys
import os
from pathlib import Path

# Basisverzeichnis (JupyterHub/Standard-Jupyter)
BASE_DIR = Path("./triple-jupyterhub").resolve()
SRC_DIR = BASE_DIR / "src"
OUTPUT_JSON_DIR = BASE_DIR / "output_json"
OUTPUT_PLAINTEXT_DIR = BASE_DIR / "output_plaintext"
LOGS_DIR = BASE_DIR / "logs"
UPLOADS_DIR = BASE_DIR / "uploads"
GRAPHS_DIR = BASE_DIR / "graphs"

for d in [SRC_DIR, OUTPUT_JSON_DIR, OUTPUT_PLAINTEXT_DIR, LOGS_DIR, UPLOADS_DIR, GRAPHS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Abhängigkeiten prüfen (Installation bitte über Environment/requirements)
missing = []
try:
    import yaml  # pyyaml
except ImportError:
    missing.append("pyyaml")
try:
    import requests
except ImportError:
    missing.append("requests")
try:
    import plotly
except ImportError:
    missing.append("plotly")
try:
    import networkx
except ImportError:
    missing.append("networkx")
try:
    import kaleido
except ImportError:
    missing.append("kaleido")

if missing:
    print("⚠ Fehlende Pakete:", ", ".join(missing))
    print("   Bitte installiere sie im JupyterHub-Environment (z.B. requirements.txt oder Admin-Setup).")
else:
    print("✓ Alle benötigten Pakete sind verfügbar")

# Python-Path erweitern
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# file_client.py erstellen
file_client_code = '''"""Datei-Client für XML/TXT-Dateien mit TEI-Optimierung."""
import os
import re
import copy
import xml.etree.ElementTree as ET
from typing import Optional, List
import logging

logger = logging.getLogger(__name__)

class FileClient:
    """Client zum Lesen von XML/TXT-Dateien mit TEI-Optimierung."""
    
    # Verarbeitungsmodus: 'plaintext', 'raw_xml', 'xml_to_plaintext'
    processing_mode = 'xml_to_plaintext'
    # Tags die komplett entfernt werden sollen (mit Inhalt)
    exclude_tags = []
    # Tag-Attribute-Kombinationen die entfernt werden sollen z.B. [('div', 'type', 'comment')]
    exclude_tag_attrs = []
    
    def __init__(self, file_path: str):
        self.file_path = file_path
        self.file_extension = os.path.splitext(file_path)[1].lower()
    
    def read_content(self) -> Optional[str]:
        """Liest den Dateiinhalt und extrahiert relevanten Text."""
        try:
            if self.file_extension == ".xml":
                return self._read_xml()
            elif self.file_extension == ".txt":
                return self._read_txt()
            else:
                logger.warning(f"Nicht unterstütztes Dateiformat: {self.file_extension}")
                return None
        except Exception as e:
            logger.error(f"Fehler beim Lesen von {self.file_path}: {e}")
            return None
    
    def _read_xml(self) -> Optional[str]:
        """Liest XML-Datei basierend auf processing_mode."""
        # Modus: Raw XML - unverarbeitet zurückgeben
        if FileClient.processing_mode == 'raw_xml':
            with open(self.file_path, "r", encoding="utf-8") as f:
                return f.read()
        
        # XML parsen
        try:
            tree = ET.parse(self.file_path)
            root = tree.getroot()
        except ET.ParseError as e:
            logger.error(f"XML-Parse-Fehler in {self.file_path}: {e}")
            return None
        
        # Modus: XML zu Plaintext konvertieren (mit Tag-Filterung)
        namespaces = {"tei": "http://www.tei-c.org/ns/1.0"}
        
        # Arbeite mit Kopie um Original nicht zu verändern
        root_copy = copy.deepcopy(root)
        
        # Entferne ausgeschlossene Tags
        self._remove_excluded_elements(root_copy, namespaces)
        
        # Body finden und Text extrahieren
        body = root_copy.find(".//tei:body", namespaces)
        if body is None:
            body = root_copy.find(".//body")
        if body is None:
            body = root_copy
        
        return self._extract_text_from_element(body)
    
    def _remove_excluded_elements(self, root: ET.Element, namespaces: dict):
        """Entfernt ausgeschlossene Elemente aus dem XML-Baum."""
        # Entferne Tags mit bestimmten Attributen (z.B. div[@type='comment'])
        for tag, attr_name, attr_value in FileClient.exclude_tag_attrs:
            # Mit TEI-Namespace
            for elem in root.findall(f".//tei:{tag}[@{attr_name}]", namespaces):
                if attr_value in elem.get(attr_name, ''):
                    parent = self._find_parent(root, elem)
                    if parent is not None:
                        # Tail-Text bewahren
                        if elem.tail:
                            prev = list(parent).index(elem)
                            if prev > 0:
                                list(parent)[prev-1].tail = (list(parent)[prev-1].tail or '') + elem.tail
                            else:
                                parent.text = (parent.text or '') + elem.tail
                        parent.remove(elem)
            # Ohne Namespace
            for elem in root.findall(f".//{tag}[@{attr_name}]"):
                if attr_value in elem.get(attr_name, ''):
                    parent = self._find_parent(root, elem)
                    if parent is not None:
                        if elem.tail:
                            prev = list(parent).index(elem)
                            if prev > 0:
                                list(parent)[prev-1].tail = (list(parent)[prev-1].tail or '') + elem.tail
                            else:
                                parent.text = (parent.text or '') + elem.tail
                        parent.remove(elem)
        
        # Entferne komplette Tags (mit Inhalt)
        for tag in FileClient.exclude_tags:
            # Mit TEI-Namespace
            for elem in root.findall(f".//tei:{tag}", namespaces):
                parent = self._find_parent(root, elem)
                if parent is not None:
                    if elem.tail:
                        prev = list(parent).index(elem)
                        if prev > 0:
                            list(parent)[prev-1].tail = (list(parent)[prev-1].tail or '') + elem.tail
                        else:
                            parent.text = (parent.text or '') + elem.tail
                    parent.remove(elem)
            # Ohne Namespace
            for elem in root.findall(f".//{tag}"):
                parent = self._find_parent(root, elem)
                if parent is not None:
                    if elem.tail:
                        prev = list(parent).index(elem)
                        if prev > 0:
                            list(parent)[prev-1].tail = (list(parent)[prev-1].tail or '') + elem.tail
                        else:
                            parent.text = (parent.text or '') + elem.tail
                    parent.remove(elem)
    
    def _find_parent(self, root: ET.Element, target: ET.Element) -> Optional[ET.Element]:
        """Findet das Elternelement eines Elements."""
        for parent in root.iter():
            for child in parent:
                if child is target:
                    return parent
        return None
    
    def _read_txt(self) -> Optional[str]:
        """Liest TXT-Datei."""
        with open(self.file_path, "r", encoding="utf-8") as f:
            return f.read()
    
    def _extract_text_from_element(self, element) -> str:
        """Extrahiert rekursiv Text aus XML-Element."""
        texts = []
        if element.text:
            texts.append(element.text.strip())
        for child in element:
            texts.append(self._extract_text_from_element(child))
            if child.tail:
                texts.append(child.tail.strip())
        result = " ".join(filter(None, texts))
        # Bereinige mehrfache Leerzeichen
        result = re.sub(r'\\s+', ' ', result)
        return result.strip()
'''

with open(SRC_DIR / 'file_client.py', 'w', encoding='utf-8') as f:
    f.write(file_client_code)

# openwebui_client.py erstellen
openwebui_client_code = '''"""OpenWebUI-kompatibler Client mit Gemini/OpenAI-Support."""
import requests
import time
import logging
from typing import Optional

logger = logging.getLogger(__name__)

class OpenWebUIClient:
    """Client für OpenAI-kompatible APIs (inkl. Gemini)."""
    
    def __init__(self, api_key: str, base_url: str, model: str, temperature: float = 0.1):
        self.api_key = api_key
        self.base_url = base_url.rstrip("/")
        self.model = model
        self.temperature = temperature
        self.max_retries = 3
        self.retry_delay = 2
    
    def generate_response(self, prompt: str, system_prompt: Optional[str] = None) -> Optional[str]:
        """Generiert Antwort mit automatischem Retry."""
        for attempt in range(self.max_retries):
            try:
                if "generativelanguage.googleapis.com" in self.base_url:
                    return self._call_gemini_api(prompt, system_prompt)
                else:
                    return self._call_openai_api(prompt, system_prompt)
            except Exception as e:
                logger.warning(f"Versuch {attempt + 1}/{self.max_retries} fehlgeschlagen: {e}")
                if attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay * (attempt + 1))
                else:
                    logger.error(f"Alle Versuche fehlgeschlagen: {e}")
                    return None
    
    def _call_gemini_api(self, prompt: str, system_prompt: Optional[str] = None) -> Optional[str]:
        """Ruft Gemini API auf."""
        url = f"{self.base_url}:generateContent?key={self.api_key}"
        payload = {
            "contents": [{"parts": [{"text": prompt}]}],
            "generationConfig": {"temperature": self.temperature}
        }
        if system_prompt:
            payload["systemInstruction"] = {"parts": [{"text": system_prompt}]}
        response = requests.post(url, json=payload, timeout=120)
        response.raise_for_status()
        data = response.json()
        return data["candidates"][0]["content"]["parts"][0]["text"]
    
    def _call_openai_api(self, prompt: str, system_prompt: Optional[str] = None) -> Optional[str]:
        """Ruft OpenAI-kompatible API auf."""
        url = f"{self.base_url}/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": prompt})
        payload = {
            "model": self.model,
            "messages": messages,
            "temperature": self.temperature
        }
        response = requests.post(url, headers=headers, json=payload, timeout=120)
        response.raise_for_status()
        data = response.json()
        return data["choices"][0]["message"]["content"]
'''

with open(SRC_DIR / 'openwebui_client.py', 'w', encoding='utf-8') as f:
    f.write(openwebui_client_code)

print("✓ Setup abgeschlossen!")
print(f"✓ Arbeitsverzeichnis: {BASE_DIR}")
print("✓ Code-Module erstellt")

In [ ]:
# API-Konfiguration
#
## Eingaben unten anpassen (API-Provider, Key, Modell, Parameter).

# Wähle deinen API-Provider:
api_provider = "Gemini (Google)"  # Eingabe: "Gemini (Google)" oder "ChatAI (AcademicCloud)"

# API-Schlüssel:
# Wichtig: niemals in öffentlichen Notebooks teilen!
api_key = ""  # Eingabe: API-Key hier einfügen

# Modell-Auswahl:
# Verfügbare Modelle (Anzeigename → Provider):
# - gemini-3-flash-preview (Gemini Standard) → Gemini (Google)
# - gemini-2.5-flash (Gemini) → Gemini (Google)
# - llama-3.3-70b-instruct (ChatAI) → ChatAI (AcademicCloud)
selected_model_name = "gemini-3-flash-preview (Gemini Standard)"  # Eingabe: Modellname aus der Liste

# Detailgrad der Extraktion:
granularity = 3  # Eingabe: 1 (kurz) bis 10 (sehr detailliert)

# Temperatur (Kreativität):
temperature = 0.3  # Eingabe: 0.0 (deterministisch) bis 1.0 (kreativ)

# Erweiterte Optionen:
show_debug_output = False  # Eingabe: True zeigt Debug-Ausgaben bei Fehlern

# Metadaten werden IMMER gespeichert
save_file_metadata = True

# Modell-Mapping
model_mapping = {
    "gemini-3-flash-preview (Gemini Standard)": {
        "name": "gemini-3-flash-preview",
        "provider": "Gemini (Google)"
    },
    "gemini-2.5-flash (Gemini)": {
        "name": "gemini-2.5-flash",
        "provider": "Gemini (Google)"
    },
    "llama-3.3-70b-instruct (ChatAI)": {
        "name": "llama-3.3-70b-instruct",
        "provider": "ChatAI (AcademicCloud)"
    }
}

# Modell-Info extrahieren
model_info = model_mapping.get(selected_model_name)
if not model_info:
    raise ValueError(f"Unbekanntes Modell: {selected_model_name}")

selected_model = model_info["name"]
model_provider = model_info["provider"]

# Prüfe ob Modell zum gewählten Provider passt
if model_provider != api_provider:
    raise ValueError(
        f"❌ Modell-Provider-Konflikt!\n\n"
        f"Das gewählte Modell '{selected_model}' gehört zu '{model_provider}',\n"
        f"aber du hast '{api_provider}' als Provider ausgewählt.\n\n"
        f"Bitte wähle entweder:\n"
        f"• Ein Modell das zu '{api_provider}' passt, oder\n"
        f"• Ändere den Provider auf '{model_provider}'"
    )

# API-Schlüssel validieren
if not api_key or len(api_key.strip()) < 10:
    raise ValueError(
        "❌ Ungültiger API-Schlüssel!\n\n"
        "Bitte gib einen gültigen API-Schlüssel ein.\n\n"
        "API-Schlüssel erhalten:\n"
        "• Gemini: https://aistudio.google.com/apikey\n"
        "• ChatAI: https://chat-ai.academiccloud.de/"
    )

# Konfiguration basierend auf Provider
if api_provider == "Gemini (Google)":
    base_url = f"https://generativelanguage.googleapis.com/v1beta/models/{selected_model}"
else:  # ChatAI
    base_url = "https://chat-ai.academiccloud.de/v1"

# Konfiguration speichern
selected_config = {
    'provider': api_provider,
    'api_key': api_key,
    'base_url': base_url,
    'model': selected_model,
    'temperature': temperature,
    'granularity': granularity,
    'include_metadata': save_file_metadata,
    'verbose': show_debug_output
}

print(f"✓ Konfiguration gespeichert")
print(f"  Provider: {api_provider}")
print(f"  Modell: {selected_model}")
print(f"  Detailgrad: {granularity}")
print(f"  Temperatur: {temperature}")

In [ ]:
# XML-Verarbeitungsoptionen
#
## Eingaben unten anpassen (Modus, ausgeschlossene Tags/Attribute).

# Verarbeitungsmodus:
xml_processing_mode = "XML zu Plaintext (empfohlen)"  # Eingabe: "Plaintext (nur .txt)", "XML unverarbeitet", "XML zu Plaintext (empfohlen)"

# Tags komplett entfernen (mit Inhalt):
exclude_tags_input = "postscript"  # Eingabe: Komma-getrennte Liste, z.B. "postscript, note, anchor"

# Tags mit bestimmten Attributen entfernen:
exclude_attrs_input = "div:type=comment"  # Eingabe: "tag:attribut=wert", z.B. "div:type=comment, div:type=apparatus"

from file_client import FileClient

# Modus setzen
mode_map = {
    "Plaintext (nur .txt)": "plaintext",
    "XML unverarbeitet": "raw_xml",
    "XML zu Plaintext (empfohlen)": "xml_to_plaintext"
}
FileClient.processing_mode = mode_map.get(xml_processing_mode, "xml_to_plaintext")

# Ausgeschlossene Tags parsen
if exclude_tags_input.strip():
    FileClient.exclude_tags = [t.strip() for t in exclude_tags_input.split(',') if t.strip()]
else:
    FileClient.exclude_tags = []

# Ausgeschlossene Tag-Attribute parsen (Format: tag:attr=value)
FileClient.exclude_tag_attrs = []
if exclude_attrs_input.strip():
    for item in exclude_attrs_input.split(','):
        item = item.strip()
        if ':' in item and '=' in item:
            tag_part, attr_part = item.split(':', 1)
            if '=' in attr_part:
                attr_name, attr_value = attr_part.split('=', 1)
                FileClient.exclude_tag_attrs.append((tag_part.strip(), attr_name.strip(), attr_value.strip()))

print(f"✓ XML-Verarbeitung konfiguriert")
print(f"  Modus: {xml_processing_mode}")
if FileClient.exclude_tags:
    print(f"  Entfernte Tags: {', '.join(FileClient.exclude_tags)}")
if FileClient.exclude_tag_attrs:
    attrs_str = ', '.join([f"<{t}[@{a}='{v}']>" for t, a, v in FileClient.exclude_tag_attrs])
    print(f"  Entfernte Elemente: {attrs_str}")

In [ ]:
# System-Prompt bearbeiten (optional)
#
## Eingabe: Setze True, um einen eigenen Prompt zu verwenden.

use_custom_prompt = False  # Eingabe: True/False

# Optional: eigenen Prompt hier einfügen (leer = Default-Prompt)
CUSTOM_PROMPT_OVERRIDE = ""  # Eingabe: eigenen Prompt hier einfügen

# Default-Prompt
DEFAULT_PROMPT = """# Prompt: Extraktion semantischer Triples aus frühneuzeitlichen Briefen  
**(LLM-optimiert, strikt themenzentriert, konzept-hierarchisch, nicht redundant)**

Du bist ein Experte für frühneuzeitliche Korrespondenz im **mitteleuropäischen Raum des späten 18. und frühen 19. Jahrhunderts (ca. 1750–1809)**.  
Deine Aufgabe ist die Extraktion **kanonischer, nicht redundanter** semantischer Triples  
(**Subjekt – Prädikat – Objekt**) aus historischen Briefen dieses Zeitraums.

Der **historische Hintergrund Mitteleuropas** (Bildungswesen, Konfessionen, soziale Ordnung, Elternautorität, Universitätskultur, Ehr- und Reputationsnormen, Briefkultur) ist **stillschweigend zu berücksichtigen**, ohne anachronistische Begriffe oder moderne Konzepte einzuführen.

Ziel ist ein **semantisch dichtes, stabiles, themenvergleichbares Ergebnis**, das sich für LLMs, Wissensgraphen und vergleichende Analysen eignet.

---

## INPUT
Du erhältst:
- `abstraktionslevel` (1–5)
- `brieftext` (vollständiger Text inkl. Briefkopf)
- TEI-XML, ignoriere den TEI-header
- ignoriere die Kommentare
- ignoriere die Editorial Notes

---

## OUTPUT (VERBINDLICH)
Gib **ausschließlich reines JSON** aus:

```json
{
  "entities": {...},
  "praedikate": {...},
  "triples": [...],
  "parameter": {...}
}
```

Keine Erklärungen, kein Markdown, keine Zusatzfelder.

---

## VERBINDLICHE NAMENSNORMALISIERUNG

Die folgenden historischen oder vollständigen Namensformen sind **immer** auf die kanonische Form zu normalisieren:

- Johann Paul Friedrich Richter → Jean Paul  
- Johann Wolfgang von Göthe → Goethe  
- Gotthold Ephraim Lessing → Lessing  
- Immanuel Kant → Kant  
- Friedrich Schiller → Schiller  

Weitere Varianten sind **analog zu vereinheitlichen**  
(bürgerlicher Vollname → etablierter Werkname).

---

## ABSTRAKTIONSLEVEL → ZIELANZAHL TRIPLES

| Level | Ziel |
|------|------|
| 1 | 1–2 Triples – Kernaussage |
| 2 | 3–5 Triples – Kernaussage + Hauptthemen |
| 3 | 8–12 Triples – Themen + Argumentstruktur |
| 4 | 12–20 Triples – thematisch relevante Details |
| 5 | 25+ Triples – implizite, klar ableitbare Bedeutungen |

---

## KERNPRINZIP: KONZEPT-HIERARCHIE STATT AKTEURS-GRAPH

Der Brief ist **primär als thematisches System** zu modellieren, nicht als Abfolge von Handlungen oder Personen.

### VERBINDLICHER MODELLIERUNGSPFAD

#### 1. OBERKONZEPTE IDENTIFIZIEREN  
Identifiziere zuerst **übergeordnete Themenfelder** (Oberkonzepte), z.B.:

- Emotion  
- Krankheit  
- Liebe  
- Freundschaft  
- Soziale Ordnung  
- Moral / Gewissen  
- Bildung  
- Lebensweg  
- Zukunft / Erwartung  

Diese werden als **Konzept-Entitäten (Typ: Konzept)** modelliert.

---

#### 2. SUBKONZEPTE ABLEITEN  
Leite daraus **inhaltlich unterscheidbare Unterthemen** ab.

Beispiele:
- Emotion → Schwermut, Angst, Hoffnung  
- Krankheit → Hypochondrie, körperliche Schwäche  
- Soziale Ordnung → elterliche Autorität, soziale Kontrolle, Ruf  

Subkonzepte sind **immer Konzepte, niemals Personenattribute**.

---

#### 3. KONZEPT–KONZEPT-BEZIEHUNGEN MODELLIEREN  
Modelliere bevorzugt Beziehungen **zwischen Ober- und Subkonzepten** oder zwischen Subkonzepten.

Beispiele:
- Schwermut ist_Subkonzept_von Emotion  
- Hypochondrie ist_Subkonzept_von Krankheit  
- Soziale_Kontrolle verschärft Emotionale_Krise  
- Krankheit beeinflusst Lebenswille  

**Mindestens 50 % aller Triples müssen Konzept↔Konzept sein.**

---

#### 4. PERSONEN NUR ALS TRÄGER (NACHRANGIG)
Personen dürfen **nur** erscheinen, wenn sie:
- Träger eines Konzepts sind
- von einem Konzept betroffen sind
- eine thematische Dynamik auslösen

Zulässige Muster:
- Krankheit betrifft Wagner  
- Emotionale_Krise betrifft Empfänger  

Unzulässig:
- Person ↔ Person ohne thematische Vermittlung

---

#### 5. ORTE NUR ALS THEMATISCHER RAHMEN (AUSNAHME)
Orte dürfen nur modelliert werden, wenn sie:
- einen thematischen Zustand rahmen oder beeinflussen
- sozial oder symbolisch relevant sind

Unzulässig:
- reine Lokalisierungen („schreibt aus X")

---

## AUFLÖSUNG VON ZUSTANDS-, ROLLEN- UND KOMPOSIT-ENTITÄTEN

Komplexe Zuschreibungen wie  
„X-Zustand von Person Y"  
dürfen **nicht** als eigene Entität stehen bleiben.

Sie sind **immer** zu zerlegen in:
1. ein **abstraktes Konzept**
2. eine **explizite Beziehung**

Beispiel:
- „Krankheitszustand Wagners"  
  → Konzept: Krankheit  
  → Triple: Krankheit betrifft Wagner

---

## REDUNDANZ-REGELN (STRIKT)

1. Keine Dubletten  
2. Keine Spiegelungen ohne Mehrwert  
3. Keine leeren Kommunikations-Triples  
4. Komplexe Sachverhalte als Ereignisknoten  
5. Verdichten statt auflisten  

---

## ENTITÄTEN

**Typen**:  
Person | Ort | Werk | Institution | Ereignis | Konzept | Zeitpunkt | Sonstiges  

**Normalisierung**:
- „ich" → Absender
- „Sie" → Empfänger
- moderne Rechtschreibung
- vollständige Datumsangaben
- historische Ortsnamen beibehalten

**IDs**: E1, E2, E3, …

---

## PRÄDIKATE

Verwende ein **kleines, stabiles Prädikatsinventar**  
(Level 3: ca. 10–16 Prädikate).

Beispiele:
- ist_Subkonzept_von
- beeinflusst
- verschärft
- beruhigt
- reguliert
- verursacht
- interpretiert_als
- betrifft
- unterliegt
- rahmt
- empfiehlt

---

## JSON-FORMAT (VERBINDLICH)

```json
{
  "entities": {
    "E1": {"label": "...", "typ": "..."}
  },
  "praedikate": {
    "P1": {"label": "...", "normalisiert_von": ["..."]}
  },
  "triples": [
    {"subjekt": "E1", "praedikat": "P1", "objekt": "E2"}
  ],
  "parameter": {
    "granularitaet": 3,
    "anzahl_triples": 10
  }
}
```

---

## START
Analysiere nun den bereitgestellten `brieftext` gemäß diesen Regeln  
und gib **ausschließlich das JSON** aus."""

if use_custom_prompt:
    CUSTOM_PROMPT = CUSTOM_PROMPT_OVERRIDE.strip() or DEFAULT_PROMPT
    if CUSTOM_PROMPT_OVERRIDE.strip():
        print("✓ Eigener Prompt aktiviert")
    else:
        print("ℹ Eigener Prompt aktiviert, aber leer – Default-Prompt wird verwendet")
else:
    CUSTOM_PROMPT = None
    print("ℹ Standard-Prompt wird verwendet")

## Dateien manuell hochladen (ohne Widgets)

1. Öffne im Jupyter-Dateibrowser den Ordner `triple-jupyterhub/uploads`.
2. Lade deine Dateien dort hoch (.xml oder .zip).
3. Wenn du ZIPs hochlädst, werden sie in der nächsten Zelle automatisch entpackt.
4. Führe danach die nächste Zelle „Dateien vorbereiten“ aus, um die Dateien zu scannen.

In [ ]:
# Dateien vorbereiten (ohne Widgets)
#
## Eingabe: Dateien manuell in den Ordner `triple-jupyterhub/uploads` legen, dann Zelle ausführen.

import os
import zipfile
import shutil
from pathlib import Path

# Upload-Verzeichnis
base_dir = Path(globals().get("BASE_DIR", Path("./triple-jupyterhub").resolve()))
upload_dir = base_dir / "uploads"
upload_dir.mkdir(parents=True, exist_ok=True)

# Globale Variable für Dateiliste
if 'uploaded_files' not in globals():
    uploaded_files = []

def reset_all():
    """Setzt alles zurück außer API-Konfiguration."""
    global uploaded_files, results, current_graph_index
    
    # Lösche Upload-Verzeichnis
    if upload_dir.exists():
        shutil.rmtree(upload_dir)
    upload_dir.mkdir(parents=True, exist_ok=True)
    
    # Lösche Ergebnisse
    output_dir = base_dir / "output_json"
    if output_dir.exists():
        for file in output_dir.iterdir():
            if file.is_file():
                file.unlink()
    
    graphs_dir = base_dir / "graphs"
    if graphs_dir.exists():
        shutil.rmtree(graphs_dir)
    graphs_dir.mkdir(parents=True, exist_ok=True)
    
    # Variablen zurücksetzen
    uploaded_files = []
    results = []
    current_graph_index = 0
    
    print("✓ Alle Dateien und Ergebnisse gelöscht - API-Konfiguration bleibt erhalten")

def safe_extract_zipfile(zip_path: Path, target_dir: Path):
    """Sichere ZIP-Extraktion (verhindert Path Traversal)."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for member in zip_ref.infolist():
            member_path = target_dir / member.filename
            target_path = member_path.resolve()
            base_path = target_dir.resolve()
            if not str(target_path).startswith(str(base_path) + os.sep) and target_path != base_path:
                raise ValueError(f"Unsichere Pfadangabe im ZIP-Archiv erkannt: {member.filename}")
        zip_ref.extractall(target_dir)

def extract_any_zips():
    """Entpackt ZIP-Dateien im Upload-Ordner."""
    for zip_path in upload_dir.rglob('*.zip'):
        print(f"📦 Entpacke {zip_path.name}...")
        safe_extract_zipfile(zip_path, upload_dir)
        zip_path.unlink(missing_ok=True)
        print(f"✓ {zip_path.name} erfolgreich entpackt")

def scan_upload_folder():
    """Scanne Upload-Ordner nach XML-Dateien."""
    global uploaded_files
    extract_any_zips()
    uploaded_files = [str(p) for p in upload_dir.rglob('*.xml')]
    if not uploaded_files:
        print("Keine Dateien hochgeladen.")
    else:
        print(f"✓ {len(uploaded_files)} XML-Datei(en):\n")
        for i, filepath in enumerate(uploaded_files, 1):
            print(f"  {i}. {Path(filepath).name}")
    return uploaded_files

# Initialer Scan
scan_upload_folder()

In [ ]:
# Verarbeitung starten
#
## Voraussetzung: API-Konfiguration und Dateiupload abgeschlossen.

import json
import time
from datetime import datetime
from pathlib import Path
from file_client import FileClient
from openwebui_client import OpenWebUIClient

# Validierung
if 'selected_config' not in globals():
    raise ValueError(
        "❌ Keine API-Konfiguration gefunden!\n\n"
        "Bitte führe zuerst die Zelle 'API-Konfiguration' aus."
    )

if 'api_key' not in globals() or not api_key:
    raise ValueError(
        "❌ Kein API-Schlüssel gefunden!\n\n"
        "Bitte führe die Zelle 'API-Konfiguration' aus und gib deinen API-Schlüssel ein."
    )

if 'uploaded_files' not in globals() or not uploaded_files:
    raise ValueError(
        "❌ Keine Dateien hochgeladen!\n\n"
        "Bitte führe zuerst die Zelle 'Dateien hochladen' aus."
    )

# Client initialisieren
client = OpenWebUIClient(
    api_key=selected_config['api_key'],
    base_url=selected_config['base_url'],
    model=selected_config['model'],
    temperature=selected_config['temperature']
 )

# System-Prompt - IMMER den DEFAULT_PROMPT verwenden, bei Custom den bearbeiteten
system_prompt = CUSTOM_PROMPT if ('use_custom_prompt' in globals() and use_custom_prompt and CUSTOM_PROMPT) else DEFAULT_PROMPT

# Prompt-Template (vereinfacht, Hauptinstruktionen sind im System-Prompt)
def create_prompt(text, granularity):
    return f"""Abstraktionslevel: {granularity}

Brieftext:
{text}"""

# Verzeichnisse
base_dir = Path(globals().get("BASE_DIR", Path("./triple-jupyterhub").resolve()))
output_dir = base_dir / "output_json"
plaintext_dir = base_dir / "output_plaintext"
output_dir.mkdir(parents=True, exist_ok=True)
plaintext_dir.mkdir(parents=True, exist_ok=True)

# Verarbeitung
results = []

print(f"Verarbeite {len(uploaded_files)} Datei(en)...\n")

for i, filepath in enumerate(uploaded_files, 1):
    filename = os.path.basename(filepath)
    print(f"[{i}/{len(uploaded_files)}] {filename}")
    
    try:
        # Zeitmessung starten
        start_time = time.time()
        
        # Text einlesen
        file_client = FileClient(filepath)
        text = file_client.read_content()
        
        if not text:
            print(f"  ⚠ Konnte Text nicht extrahieren")
            continue
        
        # Zeichenanzahl ermitteln
        char_count = len(text)
        
        # Plaintext-Datei speichern
        plaintext_filename = os.path.splitext(filename)[0] + '.txt'
        plaintext_path = plaintext_dir / plaintext_filename
        with open(plaintext_path, 'w', encoding='utf-8') as f:
            f.write(text)
        
        # API-Anfrage
        prompt = create_prompt(text, selected_config['granularity'])
        response = client.generate_response(prompt, system_prompt)
        
        if not response:
            print(f"  ❌ Keine Antwort erhalten")
            continue
        
        # JSON parsen
        try:
            clean_response = response.strip()
            if clean_response.startswith('```'):
                clean_response = '\n'.join(clean_response.split('\n')[1:-1])
            if clean_response.startswith('json'):
                clean_response = '\n'.join(clean_response.split('\n')[1:])
            
            data = json.loads(clean_response)
            triple_count = len(data.get('triples', []))
            
            # Zeitmessung beenden
            end_time = time.time()
            execution_time = round(end_time - start_time, 2)
            
            # Metadaten im Pipeline-Format hinzufügen
            data['metadata'] = {
                'datei': os.path.splitext(filename)[0],
                'verarbeitet': datetime.now().isoformat(),
                'ausfuehrungszeit_sekunden': execution_time,
                'modell': selected_config['model'],
                'api_provider': selected_config['provider'],
                'zeichenanzahl': char_count,
                'original_text': text
            }
            
            # Ergebnis speichern
            output_filename = os.path.splitext(filename)[0] + '.json'
            output_path = output_dir / output_filename
            
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            
            results.append({
                'filename': filename,
                'triples': data.get('triples', []),
                'entities': data.get('entities', {}),
                'praedikate': data.get('praedikate', {}),
                'output_path': str(output_path),
                'plaintext_path': str(plaintext_path)
            })
            
            print(f"  ✓ {triple_count} Triples extrahiert ({execution_time}s)")
            
        except json.JSONDecodeError as e:
            print(f"  ❌ JSON-Parse-Fehler: {e}")
            if selected_config.get('verbose', False):
                print(f"  Response: {response[:200]}...")
            continue
            
    except Exception as e:
        print(f"  ❌ Fehler: {e}")
        continue

print(f"\n✓ Verarbeitung abgeschlossen!")
print(f"  {len(results)} von {len(uploaded_files)} Dateien erfolgreich verarbeitet")
total_triples = sum(len(r['triples']) for r in results)
print(f"  {total_triples} Triples gesamt extrahiert")

In [ ]:
# Ergebnis visualisieren (ohne Chrome-Abhängigkeit)
#
## Voraussetzung: Verarbeitung abgeschlossen.

import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import display, Image
from pathlib import Path
import os

if 'results' not in globals() or not results:
    print("❌ Keine Ergebnisse vorhanden. Bitte führe zuerst die Verarbeitung aus.")
else:
    # Graphs-Verzeichnis erstellen
    base_dir = Path(globals().get("BASE_DIR", Path("./triple-jupyterhub").resolve()))
    graphs_dir = base_dir / "graphs"
    graphs_dir.mkdir(parents=True, exist_ok=True)
    
    # Eingabe: Index der Datei, die angezeigt werden soll
    graph_index = 0  # Eingabe: 0 bis len(results)-1
    
    def create_graph(result, save_png=True):
        """Erstellt einen NetworkX-Graph und speichert optional als PNG."""
        triples = result['triples']
        entities = result.get('entities', {})
        praedikate = result.get('praedikate', {})
        
        if not triples:
            return None, "⚠ Keine Triples zum Visualisieren gefunden."
        
        G = nx.DiGraph()
        
        # Hilfsfunktion: Entity-ID zu Label auflösen
        def get_label(entity_id):
            if entity_id in entities:
                return entities[entity_id].get('label', entity_id)
            return entity_id
        
        # Hilfsfunktion: Prädikat-ID zu Label auflösen
        def get_predicate_label(pred_id):
            if pred_id in praedikate:
                return praedikate[pred_id].get('label', pred_id)
            return pred_id
        
        for triple in triples:
            # Unterstütze beide Formate: deutsch (subjekt/objekt) und englisch (subject/object)
            subj_id = triple.get('subjekt', triple.get('subject', ''))
            pred_id = triple.get('praedikat', triple.get('predicate', ''))
            obj_id = triple.get('objekt', triple.get('object', ''))
            
            # IDs zu Labels auflösen
            subj = get_label(subj_id)
            pred = get_predicate_label(pred_id)
            obj = get_label(obj_id)
            
            if subj and obj:
                G.add_edge(subj, obj, label=pred)
        
        if len(G.nodes()) == 0:
            return None, "⚠ Keine Knoten im Graph gefunden."
        
        pos = nx.spring_layout(G, k=2, iterations=50)
        
        if save_png:
            try:
                png_filename = os.path.splitext(result['filename'])[0] + '.png'
                png_path = graphs_dir / png_filename
                plt.figure(figsize=(12, 8))
                nx.draw_networkx(
                    G,
                    pos=pos,
                    node_size=800,
                    node_color="#b3d9ff",
                    edge_color="#888",
                    font_size=8,
                    arrows=True,
                    with_labels=True
                )
                edge_labels = nx.get_edge_attributes(G, "label")
                if edge_labels:
                    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels, font_size=7)
                plt.axis("off")
                plt.tight_layout()
                plt.savefig(png_path, dpi=150)
                plt.close()
                result['graph_path'] = str(png_path)
            except Exception as e:
                print(f"⚠ PNG-Export fehlgeschlagen: {e}")
        
        stats = f"Graph mit {len(G.nodes())} Entitäten und {len(G.edges())} Beziehungen"
        return G, stats
    
    def show_graph(index):
        """Zeigt Graph an gegebenem Index."""
        if not (0 <= index < len(results)):
            print(f"❌ Ungültiger Index: {index}. Erlaubt: 0 bis {len(results) - 1}")
            return
        
        result = results[index]
        G, stats = create_graph(result)
        
        if G:
            if 'graph_path' in result and os.path.exists(result['graph_path']):
                display(Image(filename=result['graph_path']))
            else:
                # Fallback: inline zeichnen
                pos = nx.spring_layout(G, k=2, iterations=50)
                plt.figure(figsize=(12, 8))
                nx.draw_networkx(
                    G,
                    pos=pos,
                    node_size=800,
                    node_color="#b3d9ff",
                    edge_color="#888",
                    font_size=8,
                    arrows=True,
                    with_labels=True
                )
                edge_labels = nx.get_edge_attributes(G, "label")
                if edge_labels:
                    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels, font_size=7)
                plt.axis("off")
                plt.tight_layout()
                plt.show()
            print(f"\n{stats}")
        else:
            print(stats)
    
    # Graph anzeigen
    show_graph(graph_index)

In [ ]:
# Ergebnisse als ZIP herunterladen
#
## Voraussetzung: Verarbeitung abgeschlossen.

import zipfile
from datetime import datetime
from pathlib import Path
from IPython.display import FileLink

if 'results' not in globals() or not results:
    print("❌ Keine Ergebnisse vorhanden. Bitte führe zuerst die Verarbeitung aus.")
else:
    base_dir = Path(globals().get("BASE_DIR", Path("./triple-jupyterhub").resolve()))
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    zip_filename = f'triple_extraction_results_{timestamp}.zip'
    zip_path = base_dir / zip_filename
    
    graphs_dir = base_dir / "graphs"
    
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        # JSON-Dateien hinzufügen
        for result in results:
            output_path = Path(result['output_path'])
            arcname = f"json/{output_path.name}"
            if output_path.exists():
                zipf.write(output_path, arcname)
        
        # Plaintext-Dateien hinzufügen
        for result in results:
            plaintext_path = Path(result.get('plaintext_path', ''))
            if plaintext_path.exists():
                arcname = f"plaintext/{plaintext_path.name}"
                zipf.write(plaintext_path, arcname)
        
        # PNG-Grafiken hinzufügen (falls vorhanden)
        if graphs_dir.exists():
            for result in results:
                graph_path = Path(result.get('graph_path', ''))
                if graph_path.exists():
                    arcname = f"graphs/{graph_path.name}"
                    zipf.write(graph_path, arcname)
    
    print(f"ZIP erstellt: {zip_filename}")
    print(f"  • {len(results)} JSON-Dateien")
    
    plaintext_count = sum(1 for r in results if Path(r.get('plaintext_path', '')).exists())
    if plaintext_count > 0:
        print(f"  • {plaintext_count} Plaintext-Dateien")
    
    graph_count = sum(1 for r in results if Path(r.get('graph_path', '')).exists())
    if graph_count > 0:
        print(f"  • {graph_count} PNG-Grafiken")
    
    display(FileLink(zip_path))
    print(f"\n✓ Download-Link angezeigt")

## Hilfe & Troubleshooting

| Problem | Lösung |
|---------|--------|
| **NameError: api_key not defined** | Führe die Zelle "API-Konfiguration" aus |
| **Ungültiger API-Schlüssel** | Prüfe ob der Schlüssel korrekt kopiert wurde |
| **Rate Limit Error** | Warte kurz und versuche es erneut |
| **Timeout** | Versuche es mit weniger oder kleineren Dateien |
| **Keine Triples extrahiert** | Erhöhe den Detailgrad oder passe den Prompt an |
| **JSON Parse Error** | Aktiviere "Debug-Modus" in der Konfiguration |

**API-Schlüssel erhalten:**
- Gemini: https://aistudio.google.com/apikey
- ChatAI: https://chat-ai.academiccloud.de/
- OpenAI: https://platform.openai.com/api-keys